In [1]:
import chess
import tensorflow as tf
from tensorflow import keras
import copy
import numpy as np
from stockfish import Stockfish
from IPython.display import clear_output, display

In [8]:
model = keras.models.load_model("model_large_data")
stockfish_model = Stockfish("/usr/local/Cellar/stockfish/14/bin/stockfish")

In [3]:
# Parse Dataset (PGN) to Input Matrix
inp_shape = (13, 8, 8) 

# the first index is determined by piece type
king = 0
queen = 1
rook = 2
bishop = 3
knight = 4
pawn = 5

san_translate = {
    "p": pawn,
    "n": knight,
    "b": bishop,
    "r": rook, 
    "q": queen,
    "k": king
}

# structure of matrix:
#    - White is on top (0 - 5)
#    - Black is mirrored (6 - 11)
#    - Next move is 12 (0 - black, 1 - white)

def ajust_color(index, is_white):
    if is_white:
        return index
    else:
        return inp_shape[0] - index - 1

def get_index(san):
    is_white = san.isupper()
    index = san_translate[san.lower()]
    return ajust_color(index, is_white)


def fen_to_tensor(fen, move):
    ret = np.zeros(inp_shape)
    y = 0
    for rank in fen.split("/"):
        x = 0
        for char in rank:
            if char.isnumeric():
                x += int(char)
            else:
                ret[get_index(char)][y][x] = 1
                x += 1
        y += 1
    
    ret[-1] = move
    return tf.constant(ret, dtype=tf.int8)

In [9]:
def move_to_tensor(board, move):
    board_copy = copy.deepcopy(board)
    board_copy.push_san(move)
    turn = 1 if board_copy.turn == chess.WHITE else 0
    fen = board_copy.fen()
    fen = fen[:fen.find(" ")]
    return fen_to_tensor(fen, turn)

def get_win_prob(board):
    fen = board.fen()
    fen = fen[:fen.find(" ")]
    turn = 1 if board.turn == chess.WHITE else 0
    tensor = fen_to_tensor(fen, turn)
    tensor = tf.transpose(tf.expand_dims(tensor, axis=0), [0, 2, 3, 1])
    score = float(model.predict(tensor)[0])

    side = "White"
    if score < 0:
        side = "Black"
    
    score = abs(score)

    return score, side

def sort_by_second(e):
    return float(e[1])

def get_sorted_moves(model, board):
    sans = [board.san(move) for move in board.legal_moves]
    tensors = []
    for san in sans:
        tensors.append(move_to_tensor(board, san))
    
    tensors = tf.stack(tensors)
    try:
        scores =  model.predict(tf.transpose(tensors, [0, 2, 3, 1]))
    except Exception as e:
        print("Tensors: ", tensors)
        print("Sans: ", sans)
        raise Exception(e)
    tuple_list = []
    for i, sans in enumerate(sans):
        tuple_list.append((sans, float(scores[i][0])))
    tuple_list.sort(key=sort_by_second)
    return tuple_list

def best_move(model, board, depth=1, search_per_node=3):
    best_moves = get_sorted_moves(model, board)

    if depth == 0:
        if board.turn == chess.WHITE: # 1 is white favored -1 is black favored, sorted small to large
            return best_moves[-1]
        else:
            return best_moves[0]

    if board.turn == chess.WHITE:
        best_moves.reverse()

    ret_move = None
    for move in best_moves[:search_per_node]:
        board_copy = copy.deepcopy(board)
        board_copy.push_san(move[0])
        if board_copy.is_checkmate():
            return (move[0], 1 if board.turn == chess.WHITE else -1)

        if not board_copy.is_stalemate():
            deep_best_move = best_move(model, board_copy, depth=depth-1)
        else:
            deep_best_move = (move[0], 0)

        if not ret_move or board.turn == chess.WHITE and ret_move[1] > deep_best_move[1] or board.turn == chess.BLACK and ret_move[1] < deep_best_move[1]:
            ret_move = (move[0], deep_best_move[1]) # ZBzUiLW-

    return ret_move


In [14]:
def play_chess(auto_black=True, auto_white=False, elo=6000):
    board = chess.Board()
    turns = 0
    correct_moves = 0
    # stockfish_model.set_fen_position(board.fen())
    stockfish_model.set_skill_level(2)
    while not board.is_game_over():
        nn_move = auto_black and board.turn == chess.BLACK or auto_white and board.turn == chess.WHITE
        if nn_move:
            move = best_move(model, board)[0]
        else:
            # move = input("Enter move [{}]: ".format("w" if board.turn == chess.WHITE else "b"))
            stockfish_model.set_fen_position(board.fen())
            # print(stockfish_model.get_board_visual())
            uci_move = stockfish_model.get_best_move()
            move = board.san(board.parse_uci(uci_move))

        board.push_san(move)
        turns += 1

        stockfish_model.set_fen_position(board.fen())
        if stockfish_model.is_move_correct(move) and nn_move:
            correct_moves += 1

        # clear_output(wait=True)
        # display(board)

        # stockfish_model.make_moves_from_current_position([move])

    print("Result: ", board.result())
    print("Turns: ", turns)
    print("Correct Moves: ", correct_moves)    

play_chess()

Result:  1-0
Turns:  35
Correct Moves:  0
